# Deployment and integration of Julia in production environments

### Sebastian Zając, PhD



The main goal of our part of workshop is show how we can deploy Julia app on production environments. 
 

We'll build two simple models (linear regression model and neural network) to predict median house value in the Boston suburbs. 


In the workshop we will use the dataset from [UCI repository](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/).

We use `data/housing.csv` file that stores the data.

Each record of this data base has the following fields:

* `CRIM`: per capita crime rate by town
* `ZN`: proportion of residential land zoned for lots over 25,000 sq.ft.
* `INDUS`: proportion of non-retail business acres per town
* `CHAS`: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* `NOX`: nitric oxides concentration (parts per 10 million)
* `RM`: average number of rooms per dwelling
* `AGE`: proportion of owner-occupied units built prior to 1940
* `DIS`: weighted distances to five Boston employment centres
* `RAD`: index of accessibility to radial highways
* `TAX`: full-value property-tax rate per \$10,000
* `PTRATIO`: pupil-teacher ratio by town
* `B`: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* `LSTAT`: - \% lower status of the population
* **MEDV**: - Target feature - Median value of owner-occupied homes in \$1000's

After training and evaluation, the model should be deployed to serve predictions.

The model is usually embedded into a bigger application or exposed through a web service. The mentioned solutions need additional logic to properly prepare the input data and return the prediction should be returned to the user in appropriate form.
* **JSON-based web service** - JSON payload with input observation is provided to the web service and the JSON with the prediction is returned back

## 1. Data preprocessing and model building

Model building will be proceed with 3 steps: 

1. Load data
2. Data Preprocessing (standarization)
3. Models training
4. Model saving

### 1.1 Load data

In [ ]:
]st

In [1]:
using CSV

In [2]:
using DataFrames

In [3]:
data_path = joinpath("data", "housing.csv")
houses = CSV.read(data_path, DataFrame) 

houses[1:5,:]

Row,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6
3,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33,36.2


In [ ]:
describe(houses, :min, :mean, :max, :nmissing, :eltype)

In [ ]:
# check names of our features
names(houses)

## 1.2 Preprocessing Data and pipeline

As You could learn in our workshop in many ML projects we use many kind of transformers for our data. [Standardization](https://en.wikipedia.org/wiki/Standard_score) is common preprocessing step in many Data Science projects.
By default `Standardizer` standardize only features with `Continuous` scientific type. If You need transform categorical data You can use `OneHotEncoder` transform.

Multiple preparation steps can be combined in **pipelines** by using the `Pipeline` constructor or `|>` symbol - pipelines make repetitive tasks simpler and easier to execute and save.

From production point of view You must remember what features we have in Raw Data and what are features for model preparation (before standard transformations in many cases we used feature enginiering).

One of the solution in Julia programming language is library`MLJ` which expose a common interface for managing and manipulating transformers, models and other objects.

In [4]:
using MLJ

In [5]:
preproc = Standardizer()

Standardizer(
  features = Symbol[], 
  ignore = false, 
  ordered_factor = false, 
  count = false)

In [6]:
y, X = unpack(houses, ==(:MEDV), !=(:MEDV))
# create MLJ machine     
preproc_wrapped = machine(preproc, X)
# fit_transform our data
X_prepared = MLJ.transform(fit!(preproc_wrapped), X)

X_prepared[1:5,:]

[ Info: Training machine(Standardizer(features = Symbol[], …), …).


Row,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.419367,0.284548,-1.28664,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.45756,0.440616,-1.0745
2,-0.416927,-0.48724,-0.592794,-0.272329,-0.73953,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953
3,-0.416929,-0.48724,-0.592794,-0.272329,-0.73953,1.28145,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.20753
4,-0.416338,-0.48724,-1.30559,-0.272329,-0.834458,1.0153,-0.809088,1.07667,-0.752178,-1.10502,0.11292,0.415751,-1.36017
5,-0.412074,-0.48724,-1.30559,-0.272329,-0.834458,1.22736,-0.510674,1.07667,-0.752178,-1.10502,0.11292,0.440616,-1.02549


In [8]:
fitted_params(preproc_wrapped)

(features_fit = [:CHAS, :NOX, :CRIM, :B, :RM, :PTRATIO, :RAD, :INDUS, :LSTAT, :ZN, :DIS, :AGE, :TAX],
 means = (0.0691699604743083, 0.5546950592885375, 3.613523557312253, 356.6740316205536, 6.284634387351777, 18.455533596837938, 9.549407114624506, 11.136778656126475, 12.653063241106718, 11.363636363636363, 3.795042687747036, 68.57490118577074, 408.2371541501976),
 stds = (0.25399404134041, 0.11587767566755594, 8.60154510533249, 91.29486438415783, 0.7026171434153233, 2.164945523714442, 8.707259384239372, 6.860352940897579, 7.141061511348571, 23.322452994515164, 2.105710126627611, 28.148861406903617, 168.53711605495909),)

From our perspective most important is save and load full pipeline

In [9]:
MLJ.save("preprocessing.mlj", preproc_wrapped)

let's check how to load and use it

In [10]:
preproc_load = machine("preprocessing.mlj")
# remember use restore! function 
restore!(preproc_load)

trained Machine; caches model-specific representations of data
  model: Standardizer(features = Symbol[], …)
  args: 


In [11]:
X_test = X[1:3,:]
X_test_std = MLJ.transform(preproc_load, X_test)

Row,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.419367,0.284548,-1.28664,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.45756,0.440616,-1.0745
2,-0.416927,-0.48724,-0.592794,-0.272329,-0.73953,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953
3,-0.416929,-0.48724,-0.592794,-0.272329,-0.73953,1.28145,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.20753


## 1.3 Models training

#### Let's create simple linear regression model 

More info about linear regression You can find in Day_2a_Classical_predictive_model

In [12]:
houses_std = X_prepared
houses_std[!, "MEDV"] = y
houses_std[1:4,:]

Row,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.419367,0.284548,-1.28664,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.45756,0.440616,-1.0745,24.0
2,-0.416927,-0.48724,-0.592794,-0.272329,-0.73953,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953,21.6
3,-0.416929,-0.48724,-0.592794,-0.272329,-0.73953,1.28145,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.20753,34.7
4,-0.416338,-0.48724,-1.30559,-0.272329,-0.834458,1.0153,-0.809088,1.07667,-0.752178,-1.10502,0.11292,0.415751,-1.36017,33.4


In [13]:
using GLM

In [14]:
# simple linear regression model

model_specification = @formula(MEDV ~ CRIM + INDUS + CHAS + RM + AGE + DIS + TAX + LSTAT)
linear_model = lm(model_specification, houses_std)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MEDV ~ 1 + CRIM + INDUS + CHAS + RM + AGE + DIS + TAX + LSTAT

Coefficients:
───────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)  Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  22.5328      0.231675   97.26    <1e-99  22.0776    22.988
CRIM         -0.664513    0.295568   -2.25    0.0250  -1.24523   -0.0837957
INDUS        -1.07263     0.429755   -2.50    0.0129  -1.91699   -0.228265
CHAS          0.928661    0.237252    3.91    0.0001   0.462522   1.3948
RM            3.35801     0.306851   10.94    <1e-24   2.75513    3.9609
AGE          -0.663807    0.389389   -1.70    0.0889  -1.42886    0.101243
DIS          -2.22471     0.395813   -5.62    <1e-07  -3.0

In [15]:
# take a first two row of data
test_prediction = houses_std[1:2, [:CRIM, :INDUS, :CHAS, :RM, :AGE, :DIS, :TAX, :LSTAT]]
# test_prediction = houses_std[1:2, :]

# predict value for this rows
GLM.predict(linear_model, test_prediction)

2-element Vector{Union{Missing, Float64}}:
 29.919737211857687
 25.132175864045543

In [16]:
raw_data = Dict("DIS" => 3.02,"ZN" => 6,"RAD" => 2,
                "CRIM" => 0.00532, "NOX"=> 0.52,
                "PTRATIO" => 0.11292,"INDUS" => 1.51,
                "RM" => 4.53,"AGE" => 40.2,
                "CHAS" => 0.0,"TAX" => 296.0,
                "B" => 397 ,"LSTAT" => 4.98)

Dict{String, Real} with 13 entries:
  "RAD"     => 2
  "PTRATIO" => 0.11292
  "INDUS"   => 1.51
  "B"       => 397
  "ZN"      => 6
  "DIS"     => 3.02
  "CRIM"    => 0.00532
  "RM"      => 4.53
  "AGE"     => 40.2
  "CHAS"    => 0.0
  "NOX"     => 0.52
  "TAX"     => 296.0
  "LSTAT"   => 4.98

In [17]:
GLM.predict(linear_model, DataFrame(raw_data))

1-element Vector{Union{Missing, Float64}}:
 -260.42136230083725

## what we have missed ? 

In [18]:
std_data = MLJ.transform(preproc_load, DataFrame(raw_data))

Row,AGE,B,CHAS,CRIM,DIS,INDUS,LSTAT,NOX,PTRATIO,RAD,RM,TAX,ZN
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.00803,0.441711,-0.272329,-0.419483,-0.368067,-1.40325,-1.0745,-0.299411,-8.47255,-0.867024,-2.49728,-0.665949,-0.229977


In [19]:
GLM.predict(linear_model, DataFrame(std_data))

1-element Vector{Union{Missing, Float64}}:
 21.991264610651193

> Let's think - how You can use Your great model and share this model with a colleague of yours ?

## 1.4 Model saving with BSON

In [20]:
using BSON: @save
@save "linear_model.bson" linear_model

In [21]:
using LinearAlgebra
using BSON: @load

linear_model = nothing

@load "linear_model.bson" linear_model

GLM.predict(linear_model, DataFrame(std_data))

1-element Vector{Union{Missing, Float64}}:
 21.991264610651193

### Neural network model with Flux

let's make neural network model

In [62]:
X_std = transpose(Matrix(houses_std[!,Not(:MEDV)]))
y = transpose(houses.MEDV);

In [63]:
using Flux
using ProgressMeter

# Neural network model one dense hidden layer with ReLU activation function

# data
data = [(X_std, y)]
# model type
nn_model = Chain(Dense(13 => 8, relu), Dense(8 => 1))
# loss function definition
loss(x, y) = Flux.Losses.mse(nn_model(x), y)
# hyperparams
parameters = Flux.params(nn_model)
# optymalization algorithm type
opt = Flux.Adam(0.002)

@showprogress for epoch in 1:50_000
    Flux.train!(loss, parameters, data, opt)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [64]:
first_row_nn = X_std[:,1]
println("from NN model: ", nn_model(first_row_nn)[1])

from NN model: 26.474698261931266


save neural network model

In [ ]:
using BSON: @save
@save "nn_model.bson" nn_model

check if loaded model works

In [65]:
using BSON: @load
nn_model = nothing
@load "nn_model.bson" nn_model

println("from NN model: ", nn_model(first_row_nn)[1])

from NN model: 22.878820704787696


In [66]:
X = transpose(Matrix(houses[!,Not(:MEDV)]))
y = transpose(houses.MEDV);

using Flux
using ProgressMeter

# Neural network model one dense hidden layer with ReLU activation function

# data
data = [(X, y)]
# model type
nn_model = Chain(Dense(13 => 8, relu), Dense(8 => 1))
# loss function definition
loss(x, y) = Flux.Losses.mse(nn_model(x), y)
# hyperparams
parameters = Flux.params(nn_model)
# optymalization algorithm type
opt = Flux.Adam(0.002)

@showprogress for epoch in 1:50_000
    Flux.train!(loss, parameters, data, opt)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


## 1.4.1 Simple Model evaluation

In [67]:
# model evaluation 
using Statistics

RMSE(y, ŷ) = sqrt(mean((y - ŷ).^2));

In [68]:
# for regression model
y = houses.MEDV

RMSE(y, GLM.predict(linear_model, houses_std))

5.164851246622656

In [69]:
# for neural network
y = transpose(houses.MEDV)
RMSE(y, nn_model(X))

4.67928579419243

In [70]:
RMSE(y[1], transpose(GLM.predict(linear_model, DataFrame(std_data))[1]))

2.008735389348807

In [71]:
RMSE(y[1], nn_model(first_row_nn)[1])

6.77147648339681

## 1.4.2 Prepare raw data for POST request

Saving first observation from the training dataset into `test_raw_data.json` file

In [72]:
X

13×506 transpose(::Matrix{Float64}) with eltype Float64:
   0.00632    0.02731    0.02729  …    0.06076    0.10959    0.04741
  18.0        0.0        0.0           0.0        0.0        0.0
   2.31       7.07       7.07         11.93      11.93      11.93
   0.0        0.0        0.0           0.0        0.0        0.0
   0.538      0.469      0.469         0.573      0.573      0.573
   6.575      6.421      7.185    …    6.976      6.794      6.03
  65.2       78.9       61.1          91.0       89.3       80.8
   4.09       4.9671     4.9671        2.1675     2.3889     2.505
   1.0        2.0        2.0           1.0        1.0        1.0
 296.0      242.0      242.0         273.0      273.0      273.0
  15.3       17.8       17.8      …   21.0       21.0       21.0
 396.9      396.9      392.83        396.9      393.45     396.9
   4.98       9.14       4.03          5.64       6.48       7.88

In [73]:
test_raw_data = Dict(names(houses)[begin:end-1].=> X[:,1])

Dict{String, Float64} with 13 entries:
  "RAD"     => 1.0
  "PTRATIO" => 15.3
  "DIS"     => 4.09
  "CRIM"    => 0.00632
  "INDUS"   => 2.31
  "RM"      => 6.575
  "AGE"     => 65.2
  "B"       => 396.9
  "ZN"      => 18.0
  "CHAS"    => 0.0
  "NOX"     => 0.538
  "TAX"     => 296.0
  "LSTAT"   => 4.98

In [74]:
using JSON
open("test_raw_data.json","w") do f
    JSON.print(f, Dict(names(houses)[begin:end-1] .=> X[:,1]),4)
end

In [75]:
using JSON3
JSON3.read(read("test_raw_data.json"))

JSON3.Object{Vector{UInt8}, Vector{UInt64}} with 13 entries:
  :RAD     => 1
  :PTRATIO => 15.3
  :DIS     => 4.09
  :CRIM    => 0.00632
  :INDUS   => 2.31
  :RM      => 6.575
  :AGE     => 65.2
  :B       => 396.9
  :ZN      => 18
  :CHAS    => 0
  :NOX     => 0.538
  :TAX     => 296
  :LSTAT   => 4.98

In [76]:
;more test_raw_data.json

{
    "RAD": 1.0,
    "PTRATIO": 15.3,
    "DIS": 4.09,
    "CRIM": 0.00632,
    "INDUS": 2.31,
    "RM": 6.575,
    "AGE": 65.2,
    "B": 396.9,
    "ZN": 18.0,
    "CHAS": 0.0,
    "NOX": 0.538,
    "TAX": 296.0,
    "LSTAT": 4.98
}


# 2.1 Simple REST API with Julia

[Genie](https://genieframework.com/docs/) is a full stack web framework for the Julia programming language
We can create simple API with Genie. We want json as a response
In the most easy way we can take GET method to send variables by url address. 


In [ ]:
using Genie, Genie.Renderer.Json
using Genie.Requests # for method GET and POST

route("/") do 
  (:message => "Hello Julia!") |> Json.json
end

route("/getapi", method=GET) do
  vars = getpayload()
  (:variables => vars) |> Json.json
end

#start the server - it will not block the Jupyter due to async=true
up(async=true)

After starting the server, you can use `curl` or other tool capable of sending and receiving HTTP requests to interact with the API.

In [ ]:
;curl http://localhost:8000/

In [ ]:
;curl http://localhost:8000/getapi\?\&val1=43\&val2=3

## Client.jl

In [ ]:
using HTTP
resp = HTTP.get("http://localhost:8000")
if resp.status == 200 
    println(String(resp.body)) 
else println("wrong page")
end

## Other languages in Julia (Python)

You can also use Python for simple client program

```julia
using PyCall
req = pyimport("requests")
r = req.get("http://localhost:8000")
print(r.status_code)
```

The server is running asynchronously in Jupyter. When you are finished, run the `down()` command to turn it off.

In [ ]:
down()

## 2.2 ML linear regression REST API with Julia

For simplicity we will choose just GLM model of linear regression

In [ ]:
# for deploy part 
# houses_short = houses[:, [:CRIM, :INDUS, :CHAS, :RM, :AGE, :DIS, :TAX, :LSTAT]]
# std =  Standardizer()
# preproc_deploy = machine(std, houses_short)
# X_pre_d = MLJ.transform(fit!(preproc_deploy), houses_short)
# MLJ.save("pre_glm.mlj", preproc_deploy)

In [79]:
using Genie, Genie.Requests, Genie.Renderer.Json
using BSON: @load
using GLM
using DataFrames
using LinearAlgebra
using MLJ


# load pipeline transformations and model
pip = machine("preprocessing.mlj")
restore!(pip)
@load "linear_model.bson" linear_model

route("/") do
"""<div style="white-space:pre">To receive a prediction for GLM linear model send POST request with JSON payload.

First row:
{
    "crim": 0.00632,
    "tax": 296.0,
    "chas": 0.0,
    "black": 396.9,
    "lstat": 4.98,
    "age": 65.2,
    "indus": 2.31,
    "rm": 6.575,
    "dis": 4.09,
    "zn": 18.0,
    "nox": 0.538,
    "ptratio": 15.3,
    "rad": 1.0
}</div>    
    """
    
end

route("/", method = POST) do
    input_data = jsonpayload()
     try
         pre_data = MLJ.transform(pip, DataFrame(input_data))
         (":input" => input_data, ":prediction" => GLM.predict(linear_model, pre_data)) |> Json.json
     catch e
         (:error => "Ooops! There was a problem while generating a prediction.") |> Json.json
     end
end


#start the server - it will not block the Jupyter due to async=true
up(port=8000, async=true)

┌ Info: 
└ Web Server starting at http://127.0.0.1:8000 
[ Info: Listening on: 127.0.0.1:8000, thread id: 1


Genie.Server.ServersCollection(Task (runnable) @0x00000002aa7ef260, nothing)

In [82]:
down()

[ Info: Server on 127.0.0.1:8000 closing


1-element Vector{Genie.Server.ServersCollection}:
 Genie.Server.ServersCollection(Task (failed) @0x00000002aa7ef260, nothing)

In [ ]:
raw_data = Dict("DIS" => 3.02,"ZN" => 6,"RAD" => 2,
                "CRIM" => 0.00532, "NOX"=> 0.52,
                "PTRATIO" => 0.11292,"INDUS" => 1.51,
                "RM" => 4.53,"AGE" => 40.2,
                "CHAS" => 0.0,"TAX" => 296.0,
                "B" => 397 ,"LSTAT" => 4.98)

In [80]:
;cat test_raw_data.json

{
    "RAD": 1.0,
    "PTRATIO": 15.3,
    "DIS": 4.09,
    "CRIM": 0.00632,
    "INDUS": 2.31,
    "RM": 6.575,
    "AGE": 65.2,
    "B": 396.9,
    "ZN": 18.0,
    "CHAS": 0.0,
    "NOX": 0.538,
    "TAX": 296.0,
    "LSTAT": 4.98
}


In [81]:
;curl -X POST -d @test_raw_data.json -H "Content-Type: application/json" http://localhost:8000/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   220    0     0  100   220      0     99  0:00:02  0:00:02 --:--:--    99

[{":input":{"RAD":1,"PTRATIO":15.3,"INDUS":2.31,"B":396.9,"ZN":18,"DIS":4.09,"CRIM":0.00632,"RM":6.575,"AGE":65.2,"CHAS":0,"NOX":0.538,"TAX":296,"LSTAT":4.98}},{":prediction":[29.919737211857687]}]

100   220    0     0  100   220      0     52  0:00:04  0:00:04 --:--:--    52[ Info: POST / 200
100   417    0   197  100   220     38     42  0:00:05  0:00:05 --:--:--    49
[ Info: GET / 200


## Docker container 

In [ ]:
] generate Docker

In [ ]:
;cd Docker

In [ ]:
;pwd

In [ ]:
] activate .

### i will use just simple GLM model

In [ ]:
] add "Genie" "MLJ" "BSON" "GLM" "DataFrames" "LinearAlgebra"

In [ ]:
;cd ..

Add you BSON file with model and create new app.jl file with genie server.
Remember change async setting
```julia
 up(port=8000, async=false)
```

*Preparation of this worksop has been supported by the Polish National Agency for Academic Exchange under the Strategic Partnerships programme, grant number BPI/PST/2021/1/00069/U/00001.*

![SGH & NAWA](../nawalogo.png)